# ТТВ рейтинг в течение дня
Пример расчета показателей TTV в разбивке по часу.

## Описание задачи и условий расчета
- Период: 01.09.2022 - 30.09.2022 по рабочим и выходным отдельно
- Временной интервал: 05:00-29:00 с разбивкой по часу
- ЦА: Россия 100+
- Место просмотра: Все места (дом+дача)
- Каналы: Total Channels									
- Статистики: TTV Rtg%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике типов дней недели найдем id нужных нам типов
cats.get_tv_research_day_type()

# Рабочие дни: W, выходные дни: E

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['100+'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: cube100Plus100Minus

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня: рабочие и выходные
daytype_filter = 'researchDayType IN (W, E)'

# Задаем временной интервал
time_filter = None

# Задаем ЦА: города 100+
basedemo_filter = 'cube100Plus100Minus = 1'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['TTVRtgPer']

# Указываем срезы: тип дня, час
slices = ['researchDayType', 'timeBand60']

# Задаем условия сортировки: тип дня (сначала рабочие - W, потом выходные - E), час (по возр.)
sortings = {'researchDayType':'DESC', 'timeBand60':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
    "totalType": "TotalChannels" #база расчета тотал статистик: Total Channels. Возможны опции: TotalTVSet, TotalChannelsThem
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
#меняем формат поля с временными интервалами для корректной сортировки
df = df.astype(dtype={'timeBand60':int})

# Формируем сводную таблицу: строки - интервалы; столбцы - типы дня; значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['timeBand60'], 
                        columns=['researchDayType'])
df

In [ ]:
# Поменять местами уровни в столбцах (раскомментируйте при необходимости)
#df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

#df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('06_TTVRtg%_during_the_day')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)